## Logging Configuration

In [ ]:
import logging
import sys

# Configuração básica do logging 
logging.basicConfig(
    level=logging.DEBUG,  # Define o nível mínimo de logs a serem capturados
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',  # Formato das mensagens de log
    handlers=[
        logging.FileHandler("log_analysis.log"),  # Escreve logs em um arquivo
        logging.StreamHandler(sys.stdout)  # Exibe logs no console
    ]
)

# Criação de um logger para este módulo
logger = logging.getLogger(__name__)


In [ ]:
import os
from dotenv import load_dotenv

# Carrega as variáveis de ambiente do arquivo .env
load_dotenv('./.env')
os.environ["LANGSMITH_PROJECT"] = "Demo LangSmith"

## Loading dataset

In [ ]:
import pandas as pd
from langchain.docstore.document import Document

# Criação de Log (P1)
logger = logging.getLogger(__name__)

# Carrega o dataset de logs de eventos
try:
    event_log_base = pd.read_csv("datasets/teste.csv")
    logger.info(f"Dataset carregado com sucesso. Número de registros: {len(event_log_base)}.")
except FileNotFoundError as e:
    logger.error(f"Arquivo de CSV nao encontrado: {e}")
    sys.exit(1)
except Exception as e:
    logger.error(f"Erro ao carregar dataset: {e}")
    sys.exit(1)

def create_document(row):
    """ Cria um documento a partir de uma linha do DataFrame.
    Args: 
        row (pd.Series): Linha do DataFrame contendo os dados do log de eventos.
    Returns:
        Document: Um objeto Document contendo o conteúdo e metadados do log.
    """
    try:
        content = f"""
        Machine name: {row['MachineName']}.
        Category: {row['Category']}.
        Entry type: {row['EntryType']}.
        Message: {row['Message'].lower()}.
        Source: {row['Source']}.
        Time generated: {row['TimeGenerated']}.
        """
        doc = Document(
            page_content=content, 
            metadata={
                "Machine name": row['MachineName'], 
                "Entry type": row['EntryType'], 
                "Time generated": row['TimeGenerated'],
                "Source": row['Source']
            }
        )
        logger.debug(f"Documento criado para MachineName: {row['MachineName']}, Entry Type: {row['EntryType']}.")
        return doc
    except KeyError as e:
        logger.warning(f"Chave ausente no row: {e}. Row: {row}")
        return None
    except Exception as e:
        logger.error(f"Erro ao criar documento: {e}. Row: {row}")
        return None

logger.info("Criando documentos a partir do dataset.")
docs = [create_document(row) for _, row in event_log_base.iterrows()]

# Remover documentos que não foram criados devido a erros
docs = [doc for doc in docs if doc is not None]
logger.info(f"Total de documentos criados: {len(docs)}.")

logger.debug(f"Tamanho do conteúdo do primeiro documento: {len(docs[0].page_content)} caracteres.")

## Generate

In [ ]:
from langchain_ollama.llms import OllamaLLM

# Configuração do modelo OllamaLLM
try:
    logger.info("Configurando o modelo OllamaLLM.")
    llm = OllamaLLM(model="llama3.2", temperature=0) # Ajuste a temperatura conforme necessário
    logger.info("Modelo OllamaLLM configurado com sucesso.")
except Exception as e:
    logger.error(f"Erro ao configurar o modelo OllamaLLM: {e}")
    sys.exit(1)

## Spliting and storage

### Spliting

In [ ]:
from langchain_text_splitters import SpacyTextSplitter

# Configuração do SpacyTextSplitter
try:
    logger.info("Configurando o SpacyTextSplitter.")
    text_splitter = SpacyTextSplitter(chunk_size=1024, chunk_overlap=256, pipeline='pt_core_news_sm')
    logger.info("SpacyTextSplitter configurado com sucesso.")
except Exception as e:
    logger.error(f"Erro ao configurar o SpacyTextSplitter: {e}")
    sys.exit(1)

# Dividindo os documentos em chunks
logger.info("Dividindo os documentos em chunks.")
all_splits = text_splitter.split_documents(docs)
logger.info(f"Total de chunks criados: {len(all_splits)}.")

### Embedding & storage

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

# Configuração do HuggingFaceEmbeddings e definição do dispositivo 
try:
    logger.info("Configurando o HuggingFaceEmbeddings.")
    model_name = "sentence-transformers/all-mpnet-base-v2"
    model_kwargs = {'device': 'cpu'}    
    embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)
    logger.info("HuggingFaceEmbeddings configurado com sucesso.")
except Exception as e:
    logger.error(f"Erro ao configurar HuggingFaceEmbeddings: {e}")
    sys.exit(1)
    
# Configuração do Chroma Vectorstore
try:
    logger.info(f"Criando o Chroma vectorstore a partir dos chunks.") 
    vectorstore = Chroma.from_documents(
        documents=all_splits,
        embedding=embeddings
    )
except Exception as e:
    logger.error(f"Erro ao configurar o Chroma Vectorstore: {e}")
    sys.exit(1)

In [ ]:
from langchain_community.retrievers import BM25Retriever, TFIDFRetriever

# Configuração do BM25Retriever e TFIDFRetriever
try:
    logger.info("Configurando o BM25Retriever.")
    bm25_retriever = BM25Retriever.from_documents(all_splits)
    tfidf_retriever = TFIDFRetriever.from_documents(all_splits)
    bm25_retriever.k = 7
    tfidf_retriever.k = 7
    logger.info("BM25Retriever e TFIDFRetriever configurado com sucesso.")
except Exception as e:
    logger.error(f"Erro ao configurar BM25Retriever: {e}")
    sys.exit(1)

### Retriver

In [ ]:
# # relevância marginal máxima (MMR) 
# # seleciona exemplos com base em uma combinação de quais exemplos são mais semelhantes às entradas, ao mesmo tempo em que otimiza a diversidade.
# retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 20})
# # retrieved_docs = retriever.invoke("Quais são os eventos mais recentes relacionados ao Firewall?")
# retrieved_docs = retriever.invoke("Quais logs indicam algum erro?")

# len(retrieved_docs)


In [ ]:
# from langchain.retrievers.multi_query import MultiQueryRetriever

# # Adicao de logging para a configuracao do MMR usando MultiQueryRetriever
# try:
#     logger.info("Configurando o MultiQueryRetriever com MMR.")
#     retriever_from_llm = MultiQueryRetriever.from_llm(
#         retriever=vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 20}), 
#         llm=llm
#     )
#     logger.info("MultiQueryRetriever configurado com sucesso.")
# except Exception as e:
#     logger.error(f"Erro ao configurar MultiQueryRetriever: {e}")
#     sys.exit(1)

In [ ]:

from langchain.retrievers import EnsembleRetriever

# Configuração do EnsembleRetriever combinando BM25 e TFIDF
try:
    logger.info("Configurando o EnsembleRetriever combinando BM25 e Similarity Retriever.")
    ensemble_retriever = EnsembleRetriever(
        retrievers=[bm25_retriever, tfidf_retriever], 
        weights=[0.5, 0.5]  # Ajuste o peso conforme necessário
    )
    logger.info("EnsembleRetriever configurado com sucesso.")
    retrieved_docs = ensemble_retriever.invoke("Qual foi a mensagem registrada pelo sistema no evento gerado em '2020-11-01 06:38:50' pelo 'SecurityCenter'?")
    len(retrieved_docs)
except Exception as e:
    logger.error(f"Erro ao configurar EnsembleRetriever: {e}")
    sys.exit(1)

In [ ]:
# Exibir o número de documentos recuperados e filtrar por data
print(len(retrieved_docs))
target_date = "2020-11-01 06:38:50"

for i in retrieved_docs:
    if target_date in i.metadata['Time generated']:
        print(i.page_content)

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_ollama.llms import OllamaLLM

# Definição do PromptTemplate para extração de informações relevantes
custom_prompt_template = PromptTemplate(
    input_variables=["question", "context"],
    template=(
        "Você é um assistente especialista em análise de logs de eventos de segurança. "
        "Dado o seguinte evento registrado no sistema, extraia as informações mais relevantes que respondam à pergunta.\n\n"
        "Pergunta: {question}\n\n"
        "Registro de Evento:\n{context}\n\n"
        "Extração relevante:"
    )
)

# Configuração do modelo de compressão LLM
compressor_llm = OllamaLLM(model="gemma2:2b", temperature=0.0, cache=False)

compressor = LLMChainExtractor.from_llm(compressor_llm, prompt=custom_prompt_template)

contextual_compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=ensemble_retriever)

## Prompt

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    """ Formata os documentos para o prompt.
    Args:
        docs (list of Document): Lista de documentos a serem formatados.
    Returns:
        str: String formatada contendo metadados e conteúdo dos documentos.
    """
    try:
        formatted = "\n\n".join(
            f"Metadados: {doc.metadata}\nConteúdo:\n{doc.page_content}" for doc in docs
        )
        # logger.debug("Documentos formatados para o prompt.")
        return formatted
    except Exception as e:
        logger.error(f"Erro ao formatar documentos: {e}")
        return ""

template = """Você é um assistente especialista em análise de logs de eventos de segurança. 
Utilize as informações abaixo como contexto para responder à pergunta de forma clara, precisa e completa.  
Sempre que possível, cite trechos relevantes das fontes fornecidas para embasar sua resposta.  
Se a resposta não puder ser determinada a partir do contexto, diga apenas que não sabe. Não tente inventar.

### Contexto Disponível:
{context}

### Pergunta:
{question}

### Resposta Útil:"""

# Criação do PromptTemplate para o RAG Chain
try:
    custom_rag_prompt = PromptTemplate.from_template(template)
except Exception as e:
    logger.error(f"Erro ao criar PromptTemplate: {e}")
    sys.exit(1)

# Configuração do RAG Chain
try:
    rag_chain = (
        {"context": contextual_compression_retriever | format_docs, "question": RunnablePassthrough()}
        | custom_rag_prompt
        | llm
        | StrOutputParser()
    )
except Exception as e:
    logger.error(f"Erro ao configurar RAG Chain: {e}")
    sys.exit(1)
    


In [ ]:
rag_chain.invoke("Qual foi a mensagem registrada pelo sistema no evento gerado em '2020-11-01 06:38:50' pelo 'SecurityCenter'?")

In [ ]:
rag_chain.invoke("Qual foi a mensagem e o horário de término da transação do Windows Installer para o arquivo 'c2rintloc.en-us.16.msi' com id de processo '14504' em 2020-11-13?")

In [ ]:
rag_chain.invoke("Qual é o ID do processo cliente registrado para o término da transação do Windows Installer referente ao arquivo 'c2rint64.16.msi' em '2020-10-26'?")

In [ ]:
rag_chain.invoke("Qual foi o erro identificado no evento com timestamp '2020-11-07 18:21:01', e qual o caminho do arquivo mencionado no registro correspondente?")

In [ ]:
rag_chain.invoke("Qual é a sequência de eventos relacionados ao serviço de Software Protection Platform entre '2020-11-10 16:30:50' e '2020-11-13 18:07:08', incluindo os motivos de re-agendamento e os resultados das verificações de status de licenciamento?")

In [ ]:
rag_chain.invoke("Qual problema foi identificado no evento do 'igfxCUIService2.0.0.0' registrado no timestamp '2020-11-01 06:38:49' com o Event ID '0'?")